In [1]:
%load_ext autoreload
%autoreload 2
%load_ext lab_black

In [2]:
import os
import sys
import math
import random
from itertools import product
from tqdm import tqdm
from typing import Dict, List, Set, Union, Tuple
import yaml
from ast import literal_eval

import json
import pandas as pd
import seaborn as sns

# from transformers import GPTNeoXForCausalLM, AutoTokenizer
import torch
from typing import List
from matplotlib import font_manager as fm, pyplot as plt
import numpy as np
import wandb

# import statsmodels.api as sm

In [3]:
# IMPORTANT: Run as if from project root so that imports work.
pardir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
os.chdir(pardir)
os.getcwd()

'/home/kevin/code/rycolab/context-vs-prior-finetuning'

In [4]:
from model_utils.utils import (
    construct_paths_and_dataset_kwargs,
    construct_test_results_dir,
    EvalConfig,
)
from preprocessing.dataset import load_dataset_from_path

/home/kevin/mambaforge/envs/sftcontext/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Could not find the bitsandbytes CUDA binary at PosixPath('/home/kevin/mambaforge/envs/sftcontext/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda120.so')
The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.


In [5]:
##################
### Parameters ###
##################

# Data parameters

# wandb stuff
PROJECT_NAME = "sftcontext"
GROUP_NAME = None
TAGS = ["basefakepedia", "analysis", "summarize", "across-models"]
LOG_DATASETS = True

### Construct dataframes for analysis

In [6]:
analysis_dir = "analysis/summarize/generalization_datasets"
os.makedirs(analysis_dir, exist_ok=True)

In [7]:
dataset_names = ["BaseFakepedia"]
zero_shot_evals = [
    {
        "dataset_name": "BaseFakepedia",
        "subsplit": "nodup_relpid",
        "k_demonstrations": 0,
        "context_weight_format": "instruction",
        "do_steering": False,
    },
    {
        "dataset_name": "BaseFakepedia",
        "subsplit": "nodup_relpid",
        "k_demonstrations": 0,
        "context_weight_format": "float",
        "do_steering": False,
    },
]

evals = zero_shot_evals
evals = [EvalConfig(**eval) for eval in evals]
print(evals)
subsplit_names = [
    "nodup_relpid",
]
seeds = [1, 2, 3]
train_sizes = [2048]
no_train_statuses = [False]
# no_train_statuses = [False]
peft_modules = [
    json.dumps(
        ["q_proj", "k_proj", "v_proj", "o_proj"],
        # ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
        separators=(",", ":"),
    ),
]
context_weight_formats = ["instruction", "float"]
# model_id_and_bs_and_ga_and_quantize_and_peft_tuples = [
#     ("Meta-Llama-3.1-8B-Instruct", 8, 2, None, True),
#     ("Meta-Llama-3.1-8B", 8, 2, None, True),
#     ("Meta-Llama-3.1-8B", 4, 2, None, True),
# ]

model_id_and_bs_and_ga_and_quantize_and_peft_tuples = [
    ("Meta-Llama-3.1-8B-Instruct", 8, 2, None, True),
    # ("Meta-Llama-3.1-8B", 8, 2, None, True),
    # ("Meta-Llama-3.1-8B", 4, 2, None, True),
    ("Mistral-7B-Instruct-v0.3", 8, 2, None, True),
    # ("Mistral-7B-v0.3", 4, 2, None, True),
    ("gemma-2-9b-it", 2, 8, None, True),
    # ("gemma-2-9b", 4, 2, None, True),
]

[EvalConfig(dataset_name='BaseFakepedia', subsplit='nodup_relpid', k_demonstrations=0, context_weight_format='instruction', do_steering=False), EvalConfig(dataset_name='BaseFakepedia', subsplit='nodup_relpid', k_demonstrations=0, context_weight_format='float', do_steering=False)]


In [8]:
overwrite = False
df_dict = []
p_scores_df_dict = []
metrics_dict = []

for ds in dataset_names:
    for sp in subsplit_names:
        for seed in seeds:
            for ts in train_sizes:
                for nts in no_train_statuses:
                    for pm in peft_modules:
                        for cwf in context_weight_formats:
                            for (
                                model_id,
                                bs,
                                ga,
                                quantize,
                                peft,
                            ) in model_id_and_bs_and_ga_and_quantize_and_peft_tuples:
                                dict_vals = dict(
                                    DATASET_NAME=ds,
                                    SUBSPLIT=sp,
                                    SEED=seed,
                                    TRAIN_SIZE=ts,
                                    MODEL_ID=model_id,
                                    PEFT=peft and not nts,
                                    LORA_MODULES=json.loads(pm),
                                    LOAD_IN_4BIT=(quantize == "4bit"),
                                    LOAD_IN_8BIT=(quantize == "8bit"),
                                    BATCH_SZ=bs,
                                    GRAD_ACCUM=ga,
                                    NO_TRAIN=nts,
                                    CONTEXT_WEIGHT_AT_END=False,
                                    CONTEXT_WEIGHT_FORMAT=cwf,
                                    ANSWER_FORMAT_PROMPT_POSITION="end",
                                    ADD_ANSWER_FORMAT_PROMPT=False,
                                )
                                (
                                    data_dir,
                                    input_dir,
                                    model_dir,
                                    results_dir,
                                    val_results_path,
                                    data_id,
                                    full_model_id,
                                    DATASET_KWARGS_IDENTIFIABLE,
                                    MODEL_KWARGS_IDENTIFIABLE,
                                ) = construct_paths_and_dataset_kwargs(**dict_vals)
                                for (
                                    eval_name,
                                    eval_subsplit,
                                    eval_k_demonstrations,
                                    eval_ctx_weight_format,
                                    eval_do_steering,
                                ) in evals:
                                    test_results_dir = construct_test_results_dir(
                                        base_results_dir=results_dir,
                                        subsplit=eval_subsplit,
                                        context_weight_format=eval_ctx_weight_format,
                                        eval_name=eval_name,
                                        k_demonstrations=eval_k_demonstrations,
                                        in_domain_demonstrations=False,
                                        answer_format_prompt_position=None,
                                        add_answer_format_prompt=False,
                                        do_steering=False,
                                        steering_prior_value=None,
                                        steering_context_value=None,
                                        steering_layer=None,
                                    )
                                    test_results_path = os.path.join(
                                        test_results_dir, "test.csv"
                                    )
                                    # test_pscore_results_path = os.path.join(test_results_dir, "test_pscore.csv")
                                    test_metrics_path = os.path.join(
                                        test_results_dir, "metrics.json"
                                    )
                                    test_metrics_query_only_path = os.path.join(
                                        test_results_dir, "metrics_query_only.json"
                                    )
                                    if os.path.isfile(test_results_path):
                                        if "Meta" in test_results_path:
                                            print(test_results_dir)
                                        # Load predictions/results
                                        res = pd.read_csv(
                                            test_results_path,
                                        )
                                        for k, v in dict_vals.items():
                                            if isinstance(v, list):
                                                v = [v] * len(res)
                                            res[k] = v
                                        scores: List[dict] = res.to_dict("records")
                                        df_dict += [
                                            {
                                                **dict_vals,
                                                **{
                                                    "EVAL_NAME": eval_name,
                                                    "EVAL_K_DEMONSTRATIONS": eval_k_demonstrations,
                                                    "EVAL_CTX_WEIGHT_FORMAT": eval_ctx_weight_format,
                                                    "TEACH_METHOD": (
                                                        "few_shot"
                                                        if nts
                                                        else "finetune"
                                                    ),
                                                },
                                                **d,
                                            }
                                            for d in scores
                                        ]
                                    # if os.path.isfile(test_pscore_results_path):
                                    #     # Load predictions/results
                                    #     res = pd.read_csv(
                                    #         test_pscore_results_path,
                                    #     )
                                    #     for k, v in dict_vals.items():
                                    #         if isinstance(v, list):
                                    #             v = [v] * len(res)
                                    #         res[k] = v
                                    #     p_scores: List[dict] = res.to_dict("records")
                                    #     p_scores_df_dict += [{**dict_vals, **{"EVAL_NAME": eval_name, "EVAL_K_DEMONSTRATIONS": eval_k_demonstrations, "EVAL_CTX_WEIGHT_FORMAT": eval_ctx_weight_format, "TEACH_METHOD": "few_shot" if nts else "finetune"}, **d} for d in p_scores]
                                    if os.path.isfile(test_metrics_path):
                                        # Load metrics
                                        metrics = load_dataset_from_path(
                                            test_metrics_path
                                        )
                                        if os.path.isfile(test_metrics_query_only_path):
                                            metrics_query_only = load_dataset_from_path(
                                                test_metrics_query_only_path
                                            )
                                            metrics_query_only = {
                                                f"QO_{k}": v
                                                for k, v in metrics_query_only.items()
                                            }
                                        else:
                                            metrics_query_only = {}
                                        metrics_dict += [
                                            {
                                                **dict_vals,
                                                **{
                                                    "EVAL_NAME": eval_name,
                                                    "EVAL_K_DEMONSTRATIONS": eval_k_demonstrations,
                                                    "EVAL_CTX_WEIGHT_FORMAT": eval_ctx_weight_format,
                                                    "TEACH_METHOD": (
                                                        "few_shot"
                                                        if nts
                                                        else "finetune"
                                                    ),
                                                },
                                                **metrics,
                                                **metrics_query_only,
                                            }
                                        ]

data/BaseFakepedia/BaseFakepedia_nodup_relpid-ts2048/1/models/Meta-Llama-3.1-8B-Instruct-peftq_proj_k_proj_v_proj_o_proj-bs8-ga2-cwf_instruction/results/BaseFakepedia-sp_nodup_relpid-k0_OOD-cwf_instruction
data/BaseFakepedia/BaseFakepedia_nodup_relpid-ts2048/1/models/Meta-Llama-3.1-8B-Instruct-peftq_proj_k_proj_v_proj_o_proj-bs8-ga2-cwf_instruction/results/BaseFakepedia-sp_nodup_relpid-k0_OOD-cwf_float
data/BaseFakepedia/BaseFakepedia_nodup_relpid-ts2048/1/models/Meta-Llama-3.1-8B-Instruct-peftq_proj_k_proj_v_proj_o_proj-bs8-ga2-cwf_float/results/BaseFakepedia-sp_nodup_relpid-k0_OOD-cwf_instruction
data/BaseFakepedia/BaseFakepedia_nodup_relpid-ts2048/1/models/Meta-Llama-3.1-8B-Instruct-peftq_proj_k_proj_v_proj_o_proj-bs8-ga2-cwf_float/results/BaseFakepedia-sp_nodup_relpid-k0_OOD-cwf_float
data/BaseFakepedia/BaseFakepedia_nodup_relpid-ts2048/2/models/Meta-Llama-3.1-8B-Instruct-peftq_proj_k_proj_v_proj_o_proj-bs8-ga2-cwf_instruction/results/BaseFakepedia-sp_nodup_relpid-k0_OOD-cwf_instru

In [9]:
metrics_df = pd.DataFrame(metrics_dict)
# [["EVAL_NAME", "TEACH_METHOD", "accuracy", "QO_accuracy", "pair_accuracy"]]
metrics_df.info()
metrics_df.columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 30 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   DATASET_NAME                   35 non-null     object 
 1   SUBSPLIT                       35 non-null     object 
 2   SEED                           35 non-null     int64  
 3   TRAIN_SIZE                     35 non-null     int64  
 4   MODEL_ID                       35 non-null     object 
 5   PEFT                           35 non-null     bool   
 6   LORA_MODULES                   35 non-null     object 
 7   LOAD_IN_4BIT                   35 non-null     bool   
 8   LOAD_IN_8BIT                   35 non-null     bool   
 9   BATCH_SZ                       35 non-null     int64  
 10  GRAD_ACCUM                     35 non-null     int64  
 11  NO_TRAIN                       35 non-null     bool   
 12  CONTEXT_WEIGHT_AT_END          35 non-null     bool 

Index(['DATASET_NAME', 'SUBSPLIT', 'SEED', 'TRAIN_SIZE', 'MODEL_ID', 'PEFT',
       'LORA_MODULES', 'LOAD_IN_4BIT', 'LOAD_IN_8BIT', 'BATCH_SZ',
       'GRAD_ACCUM', 'NO_TRAIN', 'CONTEXT_WEIGHT_AT_END',
       'CONTEXT_WEIGHT_FORMAT', 'ANSWER_FORMAT_PROMPT_POSITION',
       'ADD_ANSWER_FORMAT_PROMPT', 'EVAL_NAME', 'EVAL_K_DEMONSTRATIONS',
       'EVAL_CTX_WEIGHT_FORMAT', 'TEACH_METHOD', 'acc', 'context_acc',
       'context_mr', 'context_pct_other', 'overall_mr', 'overall_pct_other',
       'pair_acc', 'prior_acc', 'prior_mr', 'prior_pct_other'],
      dtype='object')

In [10]:
metrics_df_short = metrics_df[
    [
        # "DATASET_NAME",
        "SEED",
        "MODEL_ID",
        # "NO_TRAIN",
        # "EVAL_NAME",
        "EVAL_CTX_WEIGHT_FORMAT",
        "CONTEXT_WEIGHT_FORMAT",
        # "EVAL_K_DEMONSTRATIONS",
        # "TEACH_METHOD",
        # "acc",
        "pair_acc",
    ]
]
# metrics_df_short["TEACH_METHOD"] = metrics_df_short.apply(
#     lambda x: (
#         "zero shot"
#         if x["TEACH_METHOD"] == "few_shot" and x["EVAL_K_DEMONSTRATIONS"] == 0
#         else x["TEACH_METHOD"]
#     ),
#     axis=1,
# )
# metrics_df_short = metrics_df_short.rename(
#     columns={
#         "MODEL_ID": "Model",
#         "EVAL_NAME": "Test Dataset",
#         "TEACH_METHOD": "Train/Eval Setting",
#     }
# )
metrics_df_short
# metrics_df_short[]

,SEED,MODEL_ID,EVAL_CTX_WEIGHT_FORMAT,CONTEXT_WEIGHT_FORMAT,pair_acc
0,1,Meta-Llama-3.1-8B-Instruct,instruction,instruction,0.930
1,1,Meta-Llama-3.1-8B-Instruct,float,instruction,0.476
2,1,Mistral-7B-Instruct-v0.3,instruction,instruction,0.914
3,1,Mistral-7B-Instruct-v0.3,float,instruction,0.056
4,1,gemma-2-9b-it,instruction,instruction,0.842
5,1,gemma-2-9b-it,float,instruction,0.726
6,1,Meta-Llama-3.1-8B-Instruct,instruction,float,0.880
7,1,Meta-Llama-3.1-8B-Instruct,float,float,0.924
8,1,Mistral-7B-Instruct-v0.3,instruction,float,0.952
9,1,gemma-2-9b-it,instruction,float,0.850


In [11]:
metrics_instruct_only = metrics_df_short[
    metrics_df_short["MODEL_ID"] == "Meta-Llama-3.1-8B"
]
metrics_instruct_only

,SEED,MODEL_ID,EVAL_CTX_WEIGHT_FORMAT,CONTEXT_WEIGHT_FORMAT,pair_acc


In [12]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np

for model, metrics_instruct_only in metrics_df_short.groupby("MODEL_ID"):
    # Calculate mean and standard deviation of pair_acc across seeds
    grouped = metrics_instruct_only.groupby(
        ["CONTEXT_WEIGHT_FORMAT", "EVAL_CTX_WEIGHT_FORMAT"]
    )["pair_acc"]
    mean_metrics = grouped.mean().unstack()
    std_metrics = grouped.std().unstack()

    # Reorder the index and columns
    mean_metrics = mean_metrics.reindex(
        columns=["instruction", "float"], index=["float", "instruction"]
    )
    std_metrics = std_metrics.reindex(
        columns=["instruction", "float"], index=["float", "instruction"]
    )

    # Create text for the heatmap cells
    text = [
        [f"{mean:.3f} ± {std:.3f}" for mean, std in zip(row_means, row_stds)]
        for row_means, row_stds in zip(mean_metrics.values, std_metrics.values)
    ]

    # Create the heatmap using plotly
    fig = go.Figure(
        data=go.Heatmap(
            z=mean_metrics.values,
            x=["🫵", "1️⃣"],
            y=["1️⃣", "🫵"],
            hoverongaps=False,
            text=text,
            texttemplate="%{text}",
            textfont={"size": 84},
            colorscale="YlGnBu",
        )
    )

    fig.update_layout(
        # title="Confusion Matrix of Mean Pair Accuracy (± Std Dev) for Meta-Llama-3.1-8B-Instruct",
        xaxis_title="Eval Intent Format",
        yaxis_title="Train Intent Format",
        width=700,
        height=600,
        font=dict(family="Computer Modern", size=60),
    )

    fig.show()

    # Save the figure to a high resolution PNG
    fig.write_image(
        f"{model}_confusion_matrix_pair_accuracy.png", scale=4, width=1400, height=1200
    )

    # Optionally, you can also save it as an interactive HTML file
    # fig.write_html("confusion_matrix_pair_accuracy.html")

    print("Figure saved as 'confusion_matrix_pair_accuracy.png'")

Figure saved as 'confusion_matrix_pair_accuracy.png'


Figure saved as 'confusion_matrix_pair_accuracy.png'


Figure saved as 'confusion_matrix_pair_accuracy.png'
